In [ ]:
from keras.models import load_model
from keras.utils  import np_utils
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from utilities import one_hot_encoding, read_train_data, read_label_data, create_cnn_model, show_train_history

SIZE = 15000
MODEL_FOLDER = "model/"
WIDTH = 190
HEIGHT = 40
NUM_DIGIT = 5
PROCESSED_FOLDER = "processed/"
LABEL_CSV_FILE = 'label.csv'
allowedChars = 'ACDEFGHJKLNPQRTUVXYZ2346789';

In [ ]:
model = create_cnn_model(WIDTH, HEIGHT, allowedChars, NUM_DIGIT)

In [ ]:
print("Reading training data...")

train_data = read_train_data(PROCESSED_FOLDER, SIZE)
train_label = read_label_data(LABEL_CSV_FILE, allowedChars, NUM_DIGIT)

print("Reading completed")

In [ ]:
filepath = MODEL_FOLDER + "{epoch:02d}-{loss:.2f}-{val_loss:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_digit4_acc', verbose=1, save_best_only=False, mode='max')
earlystop = EarlyStopping(monitor='val_loss', patience=8, verbose=1, mode='auto')
tensorBoard = TensorBoard(log_dir = 'logs', histogram_freq = 1)
callbacks_list = [tensorBoard, earlystop, checkpoint]

In [ ]:
print('model loading...')
model = load_model("twse_cnn_model.hdf5")
print('loading completed')

In [ ]:
history = model.fit(train_data, train_label, validation_split=0.2, batch_size=50, epochs=30, verbose=1, callbacks=callbacks_list)

In [ ]:
show_train_history(history, 'digit1_acc', 'val_digit1_acc')